##### Importando as bibliotecas

In [ ]:
#libs
import psycopg2
import pandas as pd
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace

##### Criando a pasta que ira receber os arquivos parquet

In [ ]:
create_bdfs = dbutils.fs.mkdirs("/dbfs/FileStore/data")
if create_bdfs:
    print("DBFS Created")
else:
    print("DBFS creation Failed")

DBFS Created


#### 3 - Extraindo cada tabela do PostgreSQL e salvando como parquet

In [ ]:
class PostgresqlToParquet:
    def __init__(self, host:str, database_name:str, username:str, password:str, table_list:list) -> None:
        self.db_params = {
            "database": database_name,
            "user": username,
            "password": password,
            "host": host,
        }
        self.conn = psycopg2.connect(**self.db_params)
        self.table_list = table_list

    def sql_to_parquet(self) -> None:
        for table_name in self.table_list:
            query = f"SELECT * FROM {table_name}"
            df = pd.read_sql(query, self.conn)
            parquet_filename = f"/dbfs/FileStore/data/{table_name}.parquet"  # Corrected path using FileStore
            df.to_parquet(parquet_filename)
            print(f"{parquet_filename} Saved!!!")

        self.conn.close()

database = PostgresqlToParquet(
    database_name="ecom1692111647951egphgoeocortbeiy",
    username="deumjhxhzvgcoaunnugtbenu@psql-mock-database-cloud",
    password="tcinixmdjluazpkqptdhnxrh",
    host="psql-mock-database-cloud.postgres.database.azure.com",
    table_list=[
        "customers",
        "employees",
        "offices",
        "orderdetails",
        "orders",
        "payments",
        "product_lines",
        "products",
    ]
)
database.sql_to_parquet()


/databricks/python/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


/dbfs/FileStore/data/customers.parquet Saved!!!


/databricks/python/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


/dbfs/FileStore/data/employees.parquet Saved!!!


/databricks/python/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


...

/dbfs/FileStore/data/payments.parquet Saved!!!


/databricks/python/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


/dbfs/FileStore/data/product_lines.parquet Saved!!!


/databricks/python/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


/dbfs/FileStore/data/products.parquet Saved!!!


##### Observando os arquivos parquet baixados

In [ ]:
display(dbutils.fs.ls("dbfs:/FileStore/data"))


path,name,size,modificationTime
dbfs:/FileStore/data/cancelled_orders_result.parquet/,cancelled_orders_result.parquet/,0,1692223411000
dbfs:/FileStore/data/customers.parquet,customers.parquet,21620,1692236807000
dbfs:/FileStore/data/employees.parquet,employees.parquet,7146,1692236807000
dbfs:/FileStore/data/offices.parquet,offices.parquet,6944,1692236807000
dbfs:/FileStore/data/orderdetails.parquet,orderdetails.parquet,27003,1692236807000
dbfs:/FileStore/data/orders.parquet,orders.parquet,14978,1692236808000
dbfs:/FileStore/data/payments.parquet,payments.parquet,10366,1692236808000
dbfs:/FileStore/data/product_lines.parquet,product_lines.parquet,8624,1692236808000
dbfs:/FileStore/data/products.parquet,products.parquet,21468,1692236809000


#### 5 - Lendo os arquivos parque para a analise

In [ ]:
# Spark Session
spark = SparkSession.builder.appName("Question5").getOrCreate()

# Parquet para Dataframes
customers_df = spark.read.parquet("dbfs:/FileStore/data/customers.parquet")
orders_df = spark.read.parquet("dbfs:/FileStore/data/orders.parquet")
orderdetails_df = spark.read.parquet("dbfs:/FileStore/data/orderdetails.parquet")
products_df = spark.read.parquet("dbfs:/FileStore/data/products.parquet")
payments_df = spark.read.parquet("dbfs:/FileStore/data/payments.parquet")
employees_df = spark.read.parquet("dbfs:/FileStore/data/employees.parquet")
offices_df = spark.read.parquet("dbfs:/FileStore/data/offices.parquet")

##### 5.1 - Qual país possui a maior quantidade de itens cancelados?

In [ ]:
# Analise
cancelled_orders_df = customers_df.join(orders_df, "customer_number") \
    .filter(col("status") == "Cancelled") \
    .groupBy("country") \
    .count() \
    .orderBy(col("count").desc())

cancelled_orders_df.show(5)


+-----------+-----+
|    country|count|
+-----------+-----+
|New Zealand|    2|
|     Sweden|    1|
|      Spain|    1|
|        USA|    1|
|         UK|    1|
+-----------+-----+



##### 5.2 - Qual o faturamento da linha de produto mais vendido, considere como os itens Shipped, cujo o pedido foi realizado no ano de 2005?

In [ ]:
# Usando os filtros necessarios na tabela orders
shipped_2005_df = orders_df.filter((col("status") == "Shipped") & (col("order_date").between("2005-01-01", "2006-01-01")))

# Analise
product_sales_df = shipped_2005_df.join(orderdetails_df, "order_number") \
    .join(products_df, "product_code") \
    .join(payments_df, "customer_number") \
    .groupBy("product_name") \
    .agg({"amount": "sum", "order_number": "count"}) \
    .withColumnRenamed("count(order_number)", "sales_quantity") \
    .withColumnRenamed("sum(amount)", "billing") \
    .orderBy(col("sales_quantity").desc())

product_sales_df.show(5)

+--------------------+------------------+--------------+
|        product_name|           billing|sales_quantity|
+--------------------+------------------+--------------+
|1992 Ferrari 360 ...| 2686883.799999999|            59|
|1958 Chevy Corvet...| 2848354.859999999|            51|
|1954 Greyhound Sc...| 2848354.859999999|            51|
|  1970 Dodge Coronet| 2848354.859999999|            51|
|1992 Porsche Caye...|2731405.1799999992|            48|
+--------------------+------------------+--------------+
only showing top 5 rows



##### 5.3 - Nome, sobrenome e e-mail dos vendedores do Japão, o local-part do e-mail deve estar mascarado.

In [ ]:
# Analise
japan_sellers = employees_df.join(offices_df, employees_df.office_code == offices_df.office_code, "left") \
    .filter(offices_df.country == "Japan") \
    .select(
        employees_df.first_name,
        employees_df.last_name,
        regexp_replace(employees_df.email, ".*@", "*****@").alias("masked_email"),  # Extract username part of email
        offices_df.country
    )

japan_sellers.show()


+----------+---------+--------------------+-------+
|first_name|last_name|        masked_email|country|
+----------+---------+--------------------+-------+
|      Mami|    Nishi|*****@classicmode...|  Japan|
|   Yoshimi|     Kato|*****@classicmode...|  Japan|
+----------+---------+--------------------+-------+



#### 6 - Salvando os resultados em tabelas com o formato delta.

In [ ]:
cancelled_orders_df.write.format("delta").mode("overwrite").save("/dbfs/FileStore/table/cancelled_orders_result.delta")
product_sales_df.write.format("delta").mode("overwrite").save("/dbfs/FileStore/table/product_sales_result.delta")
japan_sellers.write.format("delta").mode("overwrite").save("/dbfs/FileStore/table/japan_sellers.delta")

##### Observando as tabelas salvas

In [ ]:
display(dbutils.fs.ls("/dbfs/FileStore/table"))

path,name,size,modificationTime
dbfs:/dbfs/FileStore/table/cancelled_orders_result/,cancelled_orders_result/,0,1692238528000
dbfs:/dbfs/FileStore/table/cancelled_orders_result.delta/,cancelled_orders_result.delta/,0,1692238673000
dbfs:/dbfs/FileStore/table/japan_sellers/,japan_sellers/,0,1692238539000
dbfs:/dbfs/FileStore/table/japan_sellers.delta/,japan_sellers.delta/,0,1692238679000
dbfs:/dbfs/FileStore/table/product_sales_result/,product_sales_result/,0,1692238535000
dbfs:/dbfs/FileStore/table/product_sales_result.delta/,product_sales_result.delta/,0,1692238676000


##### Conferindo se os dados estao corretos

In [ ]:
# Lendo as tabelas
cancelled_orders_df = spark.read.format("delta").load("/dbfs/FileStore/table/cancelled_orders_result.delta")
product_sales_df = spark.read.format("delta").load("/dbfs/FileStore/table/product_sales_result.delta")
japan_sellers_df = spark.read.format("delta").load("/dbfs/FileStore/table/japan_sellers.delta")

cancelled_orders_df.show(5)
product_sales_df.show(5)
japan_sellers_df.show(5)

spark.stop()

+-----------+-----+
|    country|count|
+-----------+-----+
|New Zealand|    2|
|     Sweden|    1|
|      Spain|    1|
|        USA|    1|
|         UK|    1|
+-----------+-----+

+--------------------+------------------+--------------+
|        product_name|           billing|sales_quantity|
+--------------------+------------------+--------------+
|1992 Ferrari 360 ...| 2686883.799999999|            59|
|1958 Chevy Corvet...| 2848354.859999999|            51|
|1954 Greyhound Sc...| 2848354.859999999|            51|
|  1970 Dodge Coronet| 2848354.859999999|            51|
|1992 Porsche Caye...|2731405.1799999992|            48|
|Diamond T620 Semi...|        2200275.07|            41|
|     1982 Camaro Z28|2107715.2100000004|            38|
|1950's Chicago Su...|        2111090.83|            38|
|1998 Chrysler Ply...|        2155687.22|            38|
|   1957 Chevy Pickup|        2155687.22|            38|
|1962 Volkswagen M...|        2155687.22|            38|
|1964 Mercedes Tou...